The objective of this notebook is to process TED talks' metadata and transcripts extracted in the notebook *02B_Collect_metadata* for missing values and correct data types to prepare data that can be used for modelling.

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

import warnings
warnings.filterwarnings("ignore")

pd.options.display.float_format = "{:.3f}".format

%config InlineBackend.figure_format = "retina"
%matplotlib inline

## Load TED talks data

In [2]:
ted_talks = pd.read_csv("../data/ted_talks.csv")

In [3]:
# observe top 5 rows of df
display(ted_talks.head())

# no. of talks
print("Number of talks: ", ted_talks.shape[0])

# no. of columns
print("Number of columns: ", ted_talks.shape[1])

,id,title,description,url,num_views,num_comments,is_featured,video_type,event,institute_name,...,speaker_what_others_say_5,speaker_who_they_are_5,speaker_why_listen_5,speaker_id_6,speaker_name_6,speaker_description_6,speaker_is_published_6,speaker_what_others_say_6,speaker_who_they_are_6,speaker_why_listen_6
0,65059,How do our brains process speech?,"The average 20-year-old knows between 27,000 a...",https://www.ted.com/talks/gareth_gaskell_how_d...,242823,nan,False,TED-Ed Original,TED-Ed,NaN,...,nan,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN
1,65073,Give yourself permission to be creative,"Reflecting on moments that shaped his life, ac...",https://www.ted.com/talks/ethan_hawke_give_you...,685370,34.000,True,TED Stage Talk,TED2020,NaN,...,nan,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN
2,65061,How caffeine and alcohol affect your sleep,"Caffeine wakes you up, and alcohol makes you n...",https://www.ted.com/talks/matt_walker_how_caff...,90336,5.000,True,Original Content,Sleeping with Science,NaN,...,nan,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN
3,65280,"The myth of Jason, Medea, and the Golden Fleece","In Colchis, the hide of a mystical flying ram ...",https://www.ted.com/talks/iseult_gillespie_the...,221929,nan,False,TED-Ed Original,TED-Ed,NaN,...,nan,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN
4,65074,"A comprehensive, neighborhood-based response t...",Crisis interventions often focus on a single a...,https://www.ted.com/talks/kwame_owusu_kesse_a_...,579269,8.000,True,TED Stage Talk,TED2020,NaN,...,nan,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN


Number of talks:  4045
Number of columns:  78


## 3.1 Data Cleaning

In [4]:
ted_talks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4045 entries, 0 to 4044
Data columns (total 78 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         4045 non-null   int64  
 1   title                      4045 non-null   object 
 2   description                4045 non-null   object 
 3   url                        4045 non-null   object 
 4   num_views                  4045 non-null   int64  
 5   num_comments               3349 non-null   float64
 6   is_featured                4045 non-null   bool   
 7   video_type                 4045 non-null   object 
 8   event                      4045 non-null   object 
 9   institute_name             159 non-null    object 
 10  salon_name                 55 non-null     object 
 11  tags                       4045 non-null   object 
 12  num_tags                   4045 non-null   int64  
 13  more_resources             2049 non-null   objec

### 3.1a Investigate missing values

In [5]:
# function to fill missing values
def fill_missing(list_of_cols, fill_value, df=ted_talks):
    for col in list_of_cols:
        df[col] = df[col].fillna(fill_value)

In [6]:
# num_comments

# nan relates to talks which have no comments
# replace with 0
fill_missing(["num_comments"], 0)

In [7]:
# institute_name, salon_name

# view count of video types
print(ted_talks["video_type"].value_counts())

# nan relates to video types that are not TED Institute Talk/TED Salon Talk
# thus institute & salon names are not applicable
# replace with "None"
fill_missing(["institute_name", "salon_name"], "None")

TED Stage Talk              2427
TEDx Talk                    702
TED-Ed Original              590
TED Institute Talk           159
Original Content              65
TED Salon Talk (partner)      55
Best of Web                   45
Custom sponsored content       2
Name: video_type, dtype: int64


In [8]:
# more_resources, take_action, recommendations

# nan relates to talks for which these features were not included 
# replace with "None"
fill_missing(["more_resources", "take_action", "recommendations"], "None")

# there are some columns with "[]"
# replace with "None"
for i in ["more_resources", "take_action", "recommendations"]:
    ted_talks[i].replace("[]", "None", inplace=True)

In [9]:
# has_citations

# view unique values
print(ted_talks["has_citations"].unique())

# nan relates to talks with no citations
# replace with False
fill_missing(["has_citations"], False)

[nan True False]


In [10]:
# related_talk

# nan relates to talks with less than 2/3/4/5/6 related talks
# the values in these columns are the ids of the related talks

# list of applicable columns
related_cols = [c for c in ted_talks.columns if "related_talk" in c]

# replace with 0
fill_missing(related_cols, 0)

In [11]:
# recorded_date

# function to get only Y/M/D of recorded_date
# strips unnecessary trailing 0s
def get_recorded(recorded_date):
    return str(recorded_date)[:10]

# standardise datetime formats of published_time and recorded_date
ted_talks["published_time"] = pd.to_datetime(ted_talks["published_time"], unit="s")
ted_talks["recorded_date"]  = pd.to_datetime(ted_talks['recorded_date'].map(get_recorded))

# view corrected published_time and recorded_date formats
display(ted_talks[["published_time", "recorded_date"]].head())

,published_time,recorded_date
0,2020-07-23 18:37:33,2020-07-23
1,2020-07-23 14:50:06,2020-06-25
2,2020-07-22 14:12:49,2020-07-22
3,2020-07-21 15:17:29,2020-07-21
4,2020-07-21 14:54:56,2020-06-18


In [12]:
# recorded_date

# view single entry with missing recorded date
display(ted_talks.loc[ted_talks["recorded_date"].isnull()][["url", 
                                                            "title", 
                                                            "published_time", 
                                                            "recorded_date"]])

# get the median number of days between recorded_date and published_time
# used median instead of mean (approx. 308 days) as only 20% of dataset were > 300 days   
median_lead_days = (ted_talks["published_time"] - ted_talks["recorded_date"]).median()
print(f"Median days between recording & publishing: {str(median_lead_days)[:-9]}")

# replace missing recorded_date with (published_time - median_lead_days)
d = ted_talks.loc[250, "published_time"] - median_lead_days
ted_talks.loc[250, "recorded_date"] = d.date()

# view entry again to check that the correct recorded_date has been filled
display(ted_talks.loc[[250], ["url", "title", "published_time", "recorded_date"]])

,url,title,published_time,recorded_date
250,https://www.ted.com/talks/marilyn_waring_the_u...,The unpaid work that GDP ignores -- and why it...,2020-01-23 21:01:15,NaT


Median days between recording & publishing: 102 days


,url,title,published_time,recorded_date
250,https://www.ted.com/talks/marilyn_waring_the_u...,The unpaid work that GDP ignores -- and why it...,2020-01-23 21:01:15,2019-10-13


In [13]:
# speaker_name_1

# view talks with main_speaker name but nan in speaker_name_1 and speaker_id_1
display(ted_talks.loc[ted_talks["speaker_name_1"].isnull()][["title", 
                                                             "video_type", 
                                                             "main_speaker", 
                                                             "speaker_name_1", 
                                                             "speaker_id_1"]])

# nan relates to speakers who do not have a TED Speaker profile
# replace speaker_name_1 with main_speaker name
speaker_index = ted_talks.loc[ted_talks["speaker_name_1"].isnull()].index

for i in speaker_index:
    name = ted_talks.loc[i]["main_speaker"].split()
    ted_talks.loc[i, ["speaker_name_1"]] = name[1] + ", " + name[0]
    
# view entries again to check that the names of speaker_name_1 have been filled
display(ted_talks.loc[[391, 660, 670, 932], ["speaker_name_1"]])

,title,video_type,main_speaker,speaker_name_1,speaker_id_1
391,Are we living in a simulation?,TED-Ed Original,Zohreh Davoudi,NaN,nan
660,Why talent carries you further than fame,TEDx Talk,Maisie Williams,NaN,nan
670,Why we need to stop obsessing over World War II,TEDx Talk,Keith Lowe,NaN,nan
932,How prefab homes can transform affordable housing,TEDx Talk,Antón García-Abril,NaN,nan


,speaker_name_1
391,"Davoudi, Zohreh"
660,"Williams, Maisie"
670,"Lowe, Keith"
932,"García-Abril, Antón"


In [14]:
# speaker 1 (main speaker)- related attributes

# nan relates to speakers who do not have a TED Speaker profile,
# or if these attributes had not been included on the TED site

speaker1_cols = ["speaker_id_1",
                 "speaker_description_1",
                 "speaker_is_published_1",
                 "speaker_what_others_say_1",
                 "speaker_who_they_are_1",
                 "speaker_why_listen_1"]

for col in speaker1_cols:
    if col == "speaker_id_1":
        fill_missing([col], 0)
    
    if col == "speaker_is_published_1":
        fill_missing([col], False)
    
    else:
        fill_missing([col], "None")

In [15]:
# speaker 2/3/4/5/6-related attributes
# as there are more than >95% missing values in each of these columns
# drop these columns and focus on main speaker only

# get column index of speaker_id_2 as itself and columns after it will be dropped
speaker_id_2_ind = ted_talks.columns.tolist().index("speaker_id_2")

# drop columns
ted_talks.drop(ted_talks.columns[speaker_id_2_ind:], axis=1, inplace=True)

In [16]:
# transcript

# nan relates to talks with no transcript
# replace with "None"
fill_missing(["transcript"], "None")

### 3.1b Investigate data types

In [17]:
# num_comments, related_talk, speaker_id: float dtype
# is_featured, has_citations, speaker_is_published: bool dtype
# convert to integer

# create list of columns to convert to integer
cols_to_int = ted_talks.select_dtypes(include=["float64", "bool"]).columns.tolist()
cols_to_int.remove("intro_duration")

for col in cols_to_int:
    ted_talks[col] = ted_talks[col].astype(int)

In [18]:
# duration

# in seconds, convert to minutes
ted_talks["duration"] = round(ted_talks["duration"]/60, 2)

# rename column to duration_min
ted_talks.rename(columns={'duration':'duration_min'}, inplace=True)

# check that the change has been correctly effected
display(ted_talks[["duration_min"]].head())

,duration_min
0,4.350
1,9.270
2,4.900
3,4.480
4,6.600


In [19]:
# intro_duration

# view unique values, in seconds
print(ted_talks["intro_duration"].unique())

# refers to the TED intro before the actual video content is played
# for some talks, while intro_duration is 0, has an organisation/series intro
# drop column
ted_talks.drop("intro_duration", axis=1, inplace=True)

[ 0.   11.82]


In [20]:
# recorded_date

# object dtype, convert to datetime
ted_talks["recorded_date"] = pd.to_datetime(ted_talks["recorded_date"])

# check that the change has been correctly effected
display(ted_talks[["recorded_date"]].head())

,recorded_date
0,2020-07-23
1,2020-06-25
2,2020-07-22
3,2020-07-21
4,2020-06-18


In [21]:
# tags

# column value is a list stored as a str
display(ted_talks[["tags"]].head())

# extract tags from the list
ted_talks["tags"] = ted_talks["tags"].map(ast.literal_eval).str.join(",")

# check that the change has been correctly effected
display(ted_talks[["tags"]].head())

,tags
0,"['education', 'TED-Ed', 'speech', 'animation',..."
1,"['creativity', 'arts', 'life', 'humanity', 'vu..."
2,"['sleep', 'science', 'health', 'human body']"
3,"['TED-Ed', 'education', 'animation', 'history'..."
4,"['coronavirus', 'education', 'community', 'Aud..."


,tags
0,"education,TED-Ed,speech,animation,neurology,la..."
1,"creativity,arts,life,humanity,vulnerability,self"
2,"sleep,science,health,human body"
3,"TED-Ed,education,animation,history,war,violenc..."
4,"coronavirus,education,community,Audacious Proj..."


In [22]:
# take_action

# column value is a list of dictionaries stored as a str
display(ted_talks.loc[ted_talks["take_action"] != "None"][["take_action"]].head())

# function to extract list of actions
def get_actions(take_action):
    if take_action == "None":
        action_verbs = "None"
    else:
        actions = ast.literal_eval(take_action)
        action_verbs = set([actions[i]["verb"] for i in range(len(actions))])
        action_verbs = ",".join(action_verbs)
    return action_verbs

# replace take_action column with extracted action verbs
ted_talks["take_action"] = ted_talks["take_action"].map(get_actions)

# check that the change has been correctly effected
display(ted_talks.loc[ted_talks["take_action"] != "None"][["take_action"]].head())

,take_action
4,"[{'status': 'approved', 'blurb': '**Support** ..."
7,"[{'status': 'approved', 'blurb': 'When you see..."
9,"[{'status': 'approved', 'blurb': '**Donate** t..."
16,"[{'status': 'approved', 'blurb': '**Learn more..."
19,"[{'status': 'approved', 'blurb': '**Learn more..."


,take_action
4,"learn,participate"
7,participate
9,"learn,participate"
16,"follow,learn"
19,"learn,join"


In [23]:
# native_language

# view distribution of talks by native_language
print(ted_talks["native_language"].value_counts())

# talks delivered in non-english native language,
# but with an english transcript available
# focus is on talks delivered in english only

# drop tallks that are not delivered in english
non_en_index = ted_talks.loc[ted_talks["native_language"] != "en"].index
ted_talks.drop(non_en_index, axis=0, inplace=True)

# drop column & reset index
ted_talks.drop("native_language", axis=1, inplace=True)
ted_talks.reset_index(drop=True, inplace=True)

en       3996
es         28
fr          9
ja          3
hi          2
pt          1
zh-cn       1
ko          1
it          1
de          1
pt-br       1
ar          1
Name: native_language, dtype: int64


### 3.1c Re-check for missing values and data types

In [24]:
# re-check for columns with missing values
print("No. of columns with missing values: ",
      len(ted_talks.isnull().sum()[ted_talks.isnull().sum() > 0]))

No. of columns with missing values:  0


In [25]:
# re-check column data types
ted_talks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3996 entries, 0 to 3995
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         3996 non-null   int64         
 1   title                      3996 non-null   object        
 2   description                3996 non-null   object        
 3   url                        3996 non-null   object        
 4   num_views                  3996 non-null   int64         
 5   num_comments               3996 non-null   int32         
 6   is_featured                3996 non-null   int32         
 7   video_type                 3996 non-null   object        
 8   event                      3996 non-null   object        
 9   institute_name             3996 non-null   object        
 10  salon_name                 3996 non-null   object        
 11  tags                       3996 non-null   object        
 12  num_ta

## Export cleaned data as csv

In [26]:
ted_talks.to_csv("../data/ted_talks_clean.csv", index=False)